In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Project Goal and Requirements

The goal of this project is to create a basic artificial intelligence (AI) for the game of chess. This AI should use a search algorithm and an evaluation function to find an effective move in as little time as possible. The AI should be able to play both the white and the black pieces. Additionally, it should be possible for a human player to either play against this AI themselves, or set up a game where two AIs play against each other. The project should use the _python-chess_ library (see [Fiekas, N. \[2022a\]](Bibliography.ipynb#FN22a)) for basic chess functionality, including a graphical user interface (GUI).

Before we can explain how this is achieved, we must first establish some formal definitions. The implemented search algorithms, as well as other functionality, rely on these definitions.

# Two-Player Zero-Sum Games with Perfect Information

Chess is a __two-player zero-sum game with perfect information__. This means that it meets the following criteria:
1. __Two-player__: the game is played by exactly two players.
1. __Zero-sum__: one player's gain is the other player's loss (see [Wolfram Research, Inc. \[2022a\]](Bibliography.ipynb#WR22a)). In the context of chess, a move that's good for white is bad for black, and vice versa.
1. __Perfect information__: both players are completely informed about the game state and the previously performed moves (see [Wolfram Research, Inc. \[2022b\]](Bibliography.ipynb#WR22b)). In the context of chess, both the white and the black player can see the complete chess board and know which moves both of them have made. As a result, they also know what moves are available to each player on a given board state.

The following definition of a two-player zero-sum game with perfect information, from here on simply referred to as a game, is inspired by [Stroetmann, K. (2022)](Bibliography.ipynb#KS22). We define a game $\texttt{G}$ as a seven-tuple

  $$\texttt{G} = \texttt{<States, s}_0\texttt{, Moves, Players, legalMoves, finished, utility>}$$

with the following components:

1. $\texttt{States}$ is the set of possible states in the game. For chess, this is a finite set containing all possible chess board states.
1. $\texttt{s}_0 \in \texttt{States}$ is the start state. For chess, this is the default layout that the game starts in, where white can make their first move.
1. $\texttt{Moves}$ is the set of possible moves that can be made in the game. For chess, this is the set of all legal moves on all chess board states.
1. $\texttt{Players}$ is a list of length 2 that contains the players of the game. $\texttt{Players}[1]$ makes the first move. For chess, $\texttt{Players}[1] = \texttt{white}$ and $\texttt{Players}[2] = \texttt{black}$.
1. $\texttt{legalMoves}$ is a function that returns all legal moves that a player $\texttt{p} \in \texttt{Players}$ can make in a given state $\texttt{s} \in \texttt{States}$. It has the following signature:

    $$\texttt{legalMoves: States} \times \texttt{Players} \rightarrow 2^\texttt{Moves}$$

1. $\texttt{finished}$ is a function that returns whether the game is finished in a given state $\texttt{s} \in \texttt{States}$. It has the following signature:

    $$\texttt{finished: States} \rightarrow \mathbb{B}$$  

    where $\mathbb{B}$ is the set of boolean values, i.e. $\mathbb{B} := \{\texttt{True}, \texttt{False}\}$.  
    With the function $\texttt{finished}$, we can define the set $\texttt{TerminalStates}$ as the set of all states in which the game has finished. $\texttt{TerminalStates}$ is defined as follows:

    $$\texttt{TerminalStates} := \{\texttt{s} \in \texttt{States} \mid \texttt{finished}(\texttt{s})\}$$

    In the context of chess, these are all chess board states where a checkmate or a draw has been reached.

1. $\texttt{utility}$ is a function that, for a given terminal state $\texttt{s} \in \texttt{TerminalStates}$, returns the value that the game has for a player $\texttt{p} \in \texttt{Players}$, depending on the outcome of the game. It has the following signature:

    $$\texttt{utility: TerminalStates} \times \texttt{Players} \rightarrow \mathbb{Z}$$  

    If the player $\texttt{p}$ has won, $\texttt{utility}$ returns a very high positive value. If the player $\texttt{p}$ has lost, $\texttt{utility}$ returns a very high negative value. If the game has ended in a draw, $\texttt{utility}$ returns a somewhat high positive value if the player $\texttt{p}$ is in a disadvantageous position, because the draw is better than the loss they would most likely experience otherwise. If the game has ended in a draw but the player $\texttt{p}$ is in an advantageous position instead, $\texttt{utility}$ returns a somewhat high negative value, as they would be likely to win the game if they didn't take the draw.

# Heuristics

In the context of a game as defined earlier, a __heuristic__ is a function that approximates how good a move $\texttt{m} \in \texttt{Moves}$ from player $\texttt{p} \in \texttt{Players}$ in a given state $\texttt{s} \in \texttt{States}$ is. This is useful for search algorithms, which look a certain number of moves ahead and try to find the move that leads to the most advantageous position in the future. For this purpose, we also add a parameter $\texttt{o} \in \{-1, 1\}$, which is $\texttt{-1}$ if $\texttt{p}$ is the opponent player and $\texttt{1}$ otherwise. This cannot be directly derived from the parameter $\texttt{p}$, as either player may start the search, therefore the opponent player may be different between different searches.

In a complex game like chess, it is not realistically possible for these algorithms to always search all the way to a finished state. According to [Shannon, C. E. (1988)](Bibliography.ipynb#SC88), there are roughly ${10}^{43}$ legal chess positions, a number far too great for any modern computer to evaluate. Therefore, these search algorithms require a heuristic function to approximate the effectiveness of different moves and choose the best move accordingly.

For a formal definition of a heuristic, we need a function $\texttt{nextState}$ that returns the state of the game after the player $\texttt{p}$ makes the move $\texttt{m}$ in state $\texttt{s}$. It has the following signature:

$$\texttt{nextState: States} \times \texttt{Moves} \times \texttt{Players} \rightarrow \texttt{States}$$.

Now, we define a function $\texttt{heuristic}$ that approximates the effectiveness of the move $\texttt{m}$ from player $\texttt{p}$ in state $\texttt{s}$, using an additional "opponent parameter" $\texttt{o}$ as defined earlier. $\texttt{heuristic}$ has the signature:

$$\texttt{heuristic: States} \times \texttt{Moves} \times \texttt{Players} \times \{-1, 1\} \rightarrow \mathbb{Z}$$.

This function behaves differently depending on the state of the game in $\texttt{ns} := \texttt{nextState}(\texttt{s}, \texttt{m}, \texttt{p})$:  
1. If $\texttt{finished}(\texttt{ns})$, then $\texttt{heuristic}(\texttt{s}, \texttt{m}, \texttt{p}, \texttt{o}) = \texttt{utility}(\texttt{ns}, \texttt{p}) * \texttt{o}$.
1. If $\neg \texttt{finished}(\texttt{ns})$, then $\texttt{heuristic}(\texttt{s}, \texttt{m}, \texttt{p}, \texttt{o}) = \texttt{score} * \texttt{o}$, with $\texttt{score}$ being an evaluated move score for the move $\texttt{m}$ from player $\texttt{p}$ in state $\texttt{s}$.